In [1]:
from scipy.fftpack import rfft
from scipy.special import hankel2
import numpy as np
import matplotlib.pyplot as plt
# Analytical solution for the constant medium acoustic wave equation
# Given for a nx by ny domain of velocity v on a h grid with source at position xsrc,ysrc , 2second propagation at dt=1ms



T = .4
dt = .01
nt = int(T/dt+1)
nf = int(nt/2+1)
f0 = 15
tt = np.linspace(0,T,nt)
fnyq = 1. / (2*(dt))
df = 2*fnyq/nt
faxis = df*np.arange(nf)

# Ricker wavelet in frew domain
R=np.zeros((nf))
for ff in range(0,nf-1):
    R[ff]=2/np.sqrt(np.pi)*faxis[ff]**3/f0**3*np.exp(-faxis[ff]**2/f0**2)

# Analytical expression
\begin{eqnarray}
u_s(\rho, \phi, t) = \frac{1}{2\pi} \int_{-\infty}^{\infty} \{ -i \pi H_0^{(2)}\left(k | \sigma - \sigma_s| \right) F(\omega) e^{i\omega t} d\omega\}
\end{eqnarray}

In [2]:
# Ricker wavelet time domain
def ricker(f, length=0.512, dt=0.001):
    t = np.linspace(-length/2, (length-dt)/2, length/dt)
    y = (1.0 - 2.0*(np.pi**2)*(f**2)*(t**2)) * np.exp(-(np.pi**2)*(f**2)*(t**2))
    return t, y
 
f = 5 # A low wavelength of 5 Hz
t, y = ricker(f)
 
plt.plot(t, y)
plt.show()

In [3]:
nx=100
ny=100
h=5
xsrc=50
ysrc=50
v=1000


U_a=np.zeros((nx,ny,nf),dtype=complex)
for a in range(1,nf-1):
    k = 2*np.pi*faxis[a]/v
    for m in range(0,nx):
        for n in range(0,ny):
            tmp = k*np.sqrt((h*(m - xsrc))**2 + (h*(n - ysrc))**2)
            U_a[m,n,a] = -1j*np.pi* hankel2(0,tmp)*R[a]

In [4]:
# Do inverse fft on 0:dt:T (third dimension on U) and you have analytical solution o
U_t=np.zeros((nx,ny,nf))
for m in range(0,nx):
     for n in range(0,ny):
        U_t[m,n,:]=np.real(np.fft.ifft(U_a[m,n,:]))

In [5]:
from matplotlib import animation
fig = plt.figure()
plts = []             # get ready to populate this list the Line artists to be plotted
plt.hold("off")
for i in range(1,nf):
    r = plt.imshow(U_t[:,:,nf-i])   # this is how you'd plot a single line...
    plts.append( [r] )  
ani = animation.ArtistAnimation(fig, plts, interval=50,  repeat = False)   # run the animation
plt.show()